###### 1  Data Preparation before Knowledge Graph Construction

In [1]:
import pandas as pd

re_df = pd.read_csv(r'/Users/connie/Desktop/output_with_updated_ner.csv',sep=',',encoding="unicode_escape")
re_df

,Entity_A,Relation,Entity_B
0,JEFFREY P ROSEN,atty_title,Acting Attorney General
1,Kristin D Biehl,atty_title,"Supervising Deputy Attorney , General"
2,MORTON J . GORDON,atty_title,Federal Public Defender
3,MORTON J . GORDON,atty_title,"Supervising Deputy Attorney , General"
4,DANIEL P . GOLD,atty_title,Deputy Appellate Public Defender
...,...,...,...
8736,Todd A . Dawson,atty_lawFirm,"United States Attorney ' s Office , Southern D..."
8737,"United States Attorney ' s Office , Southern D...",lawFirm_city,ORISKANY FALLS
8738,ORISKANY FALLS,city_state,WI
8739,CALVINA BETTY BROWNE,atty_lawFirm,"US Department of Justice , Civil Division - Co..."


In [2]:
re_df['Entity_A_Type'] = re_df['Relation'].str.split('_',expand=True)[0]

In [3]:
re_df['Entity_B_Type'] = re_df['Relation'].str.split('_',expand=True)[1]

In [4]:
re_df

,Entity_A,Relation,Entity_B,Entity_A_Type,Entity_B_Type
0,JEFFREY P ROSEN,atty_title,Acting Attorney General,atty,title
1,Kristin D Biehl,atty_title,"Supervising Deputy Attorney , General",atty,title
2,MORTON J . GORDON,atty_title,Federal Public Defender,atty,title
3,MORTON J . GORDON,atty_title,"Supervising Deputy Attorney , General",atty,title
4,DANIEL P . GOLD,atty_title,Deputy Appellate Public Defender,atty,title
...,...,...,...,...,...
8736,Todd A . Dawson,atty_lawFirm,"United States Attorney ' s Office , Southern D...",atty,lawFirm
8737,"United States Attorney ' s Office , Southern D...",lawFirm_city,ORISKANY FALLS,lawFirm,city
8738,ORISKANY FALLS,city_state,WI,city,state
8739,CALVINA BETTY BROWNE,atty_lawFirm,"US Department of Justice , Civil Division - Co...",atty,lawFirm


In [5]:
re_dfA = re_df[['Entity_A' ,'Entity_A_Type' ]]
re_dfA.columns = ['Entity', 'Type']


In [6]:
re_dfB = re_df[['Entity_B' , 'Entity_B_Type']]
re_dfB.columns = ['Entity', 'Type']

In [7]:
 re_df_Entities = re_dfA.append(re_dfB, ignore_index = True)

In [8]:
re_df_Entities.drop_duplicates(inplace= True)
re_df_Entities = re_df_Entities.reset_index()


In [9]:
entity_id_start = 0
re_df_Entities['index'] = pd.Series(range(entity_id_start, entity_id_start + re_df_Entities.shape[0] ))

In [10]:
re_df_Entities.columns = ['Entity_Id','Entity','Type']

In [11]:
re_df_Entities['Type'].unique()

array(['atty', 'lawFirm', 'city', 'title', 'stateName', 'state'],
      dtype=object)

In [12]:
def type_convert(val):
    if val == 'atty':
        return 0
    if val == 'lawFirm':
        return 1
    if val == 'city':
        return 2
    if val == 'title':
        return 3
    if val == 'stateName':
        return 4
    if val == 'state':
        return 5
    
    

In [13]:
re_df_Entities['Type_Num'] = re_df_Entities['Type'].apply(lambda x : type_convert(x))
re_df_Entities

,Entity_Id,Entity,Type,Type_Num
0,0,JEFFREY P ROSEN,atty,0
1,1,Kristin D Biehl,atty,0
2,2,MORTON J . GORDON,atty,0
3,3,DANIEL P . GOLD,atty,0
4,4,SAMANTHA J . KATTAU,atty,0
...,...,...,...,...
3570,3570,CLARKSON,city,2
3571,3571,VAUGHNSVILLE,city,2
3572,3572,BARRETT,state,5
3573,3573,CLAYVILLE,city,2


In [15]:
re_df_Entities.to_csv(r'/Users/connie/Desktop/entity.csv',index=False)

In [16]:
#get nodes for knowledge graph construction
Entity=re_df_Entities[['Type_Num']]
Entity

,Type_Num
0,0
1,0
2,0
3,0
4,0
...,...
3570,2
3571,2
3572,5
3573,2


In [17]:
re_df_Entities2 = re_df_Entities[['Entity_Id','Entity']]


In [18]:
re_df3 = pd.merge(left=re_df, right = re_df_Entities2, how='left', left_on = "Entity_A",right_on = "Entity")

In [19]:
re_df3 = re_df3.rename(columns={'Entity_Id': 'Entity_A_Id'})
re_df3

,Entity_A,Relation,Entity_B,Entity_A_Type,Entity_B_Type,Entity_A_Id,Entity
0,JEFFREY P ROSEN,atty_title,Acting Attorney General,atty,title,0,JEFFREY P ROSEN
1,Kristin D Biehl,atty_title,"Supervising Deputy Attorney , General",atty,title,1,Kristin D Biehl
2,MORTON J . GORDON,atty_title,Federal Public Defender,atty,title,2,MORTON J . GORDON
3,MORTON J . GORDON,atty_title,"Supervising Deputy Attorney , General",atty,title,2,MORTON J . GORDON
4,DANIEL P . GOLD,atty_title,Deputy Appellate Public Defender,atty,title,3,DANIEL P . GOLD
...,...,...,...,...,...,...,...
9284,Todd A . Dawson,atty_lawFirm,"United States Attorney ' s Office , Southern D...",atty,lawFirm,658,Todd A . Dawson
9285,"United States Attorney ' s Office , Southern D...",lawFirm_city,ORISKANY FALLS,lawFirm,city,64,"United States Attorney ' s Office , Southern D..."
9286,ORISKANY FALLS,city_state,WI,city,state,3206,ORISKANY FALLS
9287,CALVINA BETTY BROWNE,atty_lawFirm,"US Department of Justice , Civil Division - Co...",atty,lawFirm,660,CALVINA BETTY BROWNE


In [20]:
re_df4 = pd.merge(left=re_df3, right = re_df_Entities2, how='left', left_on = "Entity_B",right_on = "Entity")

In [21]:
re_df4 = re_df4.rename(columns={'Entity_Id': 'Entity_B_Id'})

In [22]:
re_df4.to_csv(r'D:\GDM\Captone_Scripts\Prof_Scripts_all\OUTPUT_RE\Entity_REL_DF.csv',index=False)

In [37]:
Relation = re_df4[['Entity_A_Id' ,'Entity_B_Id' ]]
Relation=Relation.rename(columns={'Entity_B_Id': 'target'})
Relation=Relation.rename(columns={'Entity_A_Id': 'source'})
Relation.drop_duplicates(inplace= True)
Relation

,source,target
0,0,3208
1,1,3209
2,2,1596
3,2,3210
4,2,3209
...,...,...
10970,3205,3486
10971,658,64
10972,64,3206
10973,3206,3302


###### 2  Knowledge Graph Construction

In [38]:
import stellargraph
from stellargraph import StellarGraph


In [39]:
G=StellarGraph(nodes=Entity,edges=Relation)

In [40]:
print(G.info())

StellarGraph: Undirected multigraph
 Nodes: 3575, Edges: 10631

 Node types:
  default: [3575]
    Features: float32 vector, length 1
    Edge types: default-default->default

 Edge types:
    default-default->default: [10631]
        Weights: all 1 (default)
        Features: none


In [41]:
#save Neo4j graph to a pickle file
import networkx as nx

In [28]:
kg = pd.read_csv('/Users/connie/Desktop/export.csv')

In [32]:
#export visualization version of KG pickle
kg.to_pickle('/Users/connie/Desktop/kg(visual).pkl')

###### 3  Logistic regression layer construction and graph embedding preprocessing

In [54]:
#Data Preprocessing
import matplotlib.pyplot as plt
from math import isclose
from sklearn.decomposition import PCA
import os
import networkx as nx
import numpy as np
import pandas as pd
from stellargraph import StellarGraph, datasets
from stellargraph.data import EdgeSplitter
from collections import Counter
import multiprocessing
from IPython.display import display, HTML
from sklearn.model_selection import train_test_split

In [55]:
graph=G
# Define an edge splitter on the original graph:
edge_splitter_test = EdgeSplitter(graph)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from graph, and obtain the
# reduced graph graph_test with the sampled links removed:
graph_test, examples_test, labels_test = edge_splitter_test.train_test_split(
    p=0.1, method="global"
)

print(graph_test.info())

** Sampled 1063 positive and 1063 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 3575, Edges: 9568

 Node types:
  default: [3575]
    Features: float32 vector, length 1
    Edge types: default-default->default

 Edge types:
    default-default->default: [9568]
        Weights: all 1 (default)
        Features: none


In [56]:
# Do the same process to compute a training subset from within the test graph
edge_splitter_train = EdgeSplitter(graph_test, graph)
graph_train, examples, labels = edge_splitter_train.train_test_split(
    p=0.1, method="global"
)
(
    examples_train,
    examples_model_selection,
    labels_train,
    labels_model_selection,
) = train_test_split(examples, labels, train_size=0.75, test_size=0.25)

print(graph_train.info())

** Sampled 956 positive and 956 negative edges. **
StellarGraph: Undirected multigraph
 Nodes: 3575, Edges: 8612

 Node types:
  default: [3575]
    Features: float32 vector, length 1
    Edge types: default-default->default

 Edge types:
    default-default->default: [8612]
        Weights: all 1 (default)
        Features: none


In [57]:
from stellargraph.data import BiasedRandomWalk


def create_biased_random_walker(graph, walk_num, walk_length):
    # parameter settings for "p" and "q":
    p = 1.0
    q = 1.0
    return BiasedRandomWalk(graph, n=walk_num, length=walk_length, p=p, q=q)

In [58]:
walk_length = 5
epochs = 6
batch_size = 50

In [59]:
from stellargraph.data import UnsupervisedSampler
from stellargraph.mapper import Node2VecLinkGenerator, Node2VecNodeGenerator
from stellargraph.layer import Node2Vec, link_classification
from tensorflow import keras
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler


# 1. link embeddings
def link_examples_to_features(link_examples, transform_node, binary_operator):
    return [
        binary_operator(transform_node(src), transform_node(dst))
        for src, dst in link_examples
    ]


# 2. training classifier
def train_link_prediction_model(
    link_examples, link_labels, get_embedding, binary_operator
):
    clf = link_prediction_classifier()
    link_features = link_examples_to_features(
        link_examples, get_embedding, binary_operator
    )
    clf.fit(link_features, link_labels)
    return clf


def link_prediction_classifier(max_iter=5000):
    lr_clf = LogisticRegressionCV(Cs=10, cv=10, scoring="roc_auc", max_iter=max_iter)
    return Pipeline(steps=[("sc", StandardScaler()), ("clf", lr_clf)])


# 3. and 4. evaluate classifier
def evaluate_link_prediction_model(
    clf, link_examples_test, link_labels_test, get_embedding, binary_operator
):
    link_features_test = link_examples_to_features(
        link_examples_test, get_embedding, binary_operator
    )
    score = evaluate_roc_auc(clf, link_features_test, link_labels_test)
    return score


def evaluate_roc_auc(clf, link_features, link_labels):
    predicted = clf.predict_proba(link_features)

    # check which class corresponds to positive links
    positive_column = list(clf.classes_).index(1)
    return roc_auc_score(link_labels, predicted[:, positive_column])

In [60]:
def operator_hadamard(u, v):
    return u * v


def operator_l1(u, v):
    return np.abs(u - v)


def operator_l2(u, v):
    return (u - v) ** 2


def operator_avg(u, v):
    return (u + v) / 2.0


def run_link_prediction(binary_operator, embedding_train):
    clf = train_link_prediction_model(
        examples_train, labels_train, embedding_train, binary_operator
    )
    score = evaluate_link_prediction_model(
        clf,
        examples_model_selection,
        labels_model_selection,
        embedding_train,
        binary_operator,
    )
    #print("classifier: ",clf, "binary_operator: ", binary_operator,"score:", score)
    return {
        "classifier": clf,
        "binary_operator": binary_operator,
        "score": score,
    }


binary_operators = [operator_hadamard, operator_l1, operator_l2, operator_avg]

In [61]:
def train_and_evaluate(embedding, name):

    embedding_train = embedding(graph_train, "Train Graph")

    # Train the link classification model with the learned embedding
    results = [run_link_prediction(op, embedding_train) for op in binary_operators]
    best_result = max(results, key=lambda result: result["score"])
    print(
        f"\nBest result with '{name}' embeddings from '{best_result['binary_operator'].__name__}'"
    )
    display(
        pd.DataFrame(
            [(result["binary_operator"].__name__, result["score"]) for result in results],
            columns=("name", "ROC AUC"),
        ).set_index("name")
    )

    # Evaluate the best model using the test set
    test_score = evaluate_link_prediction_model(
        best_result["classifier"],
        examples_test,
        labels_test,
        embedding_train,
        best_result["binary_operator"],
    )

    return test_score

###### 4.1 Node2Vec Graph Embedding and Link Prediction 

In [62]:
def node2vec_embedding(graph, name):

    # Set the embedding dimension and walk number:
    dimension = 128
    walk_number = 20

    print(f"Training Node2Vec for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define a Node2Vec training generator, which generates batches of training pairs
    generator = Node2VecLinkGenerator(graph, batch_size)

    # Create the Node2Vec model
    node2vec = Node2Vec(dimension, generator=generator)

    # Build the model and expose input and output sockets of Node2Vec, for node pair inputs
    x_inp, x_out = node2vec.in_out_tensors()

    # Use the link_classification function to generate the output of the Node2Vec model
    prediction = link_classification(
        output_dim=1, output_act="sigmoid", edge_embedding_method="dot"
    )(x_out)

    # Stack the Node2Vec encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    model.fit(
        generator.flow(unsupervised_samples),
        epochs=epochs,
        verbose=2,
        use_multiprocessing=False,
        workers=4,
        shuffle=True,
    )

    # Build the model to predict node representations from node ids with the learned Node2Vec model parameters
    x_inp_src = x_inp[0]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

    # Get representations for all nodes in ``graph``
    node_gen = Node2VecNodeGenerator(graph, batch_size).flow(graph_node_list)
    node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

In [63]:
node2vec_result = train_and_evaluate(node2vec_embedding, "Node2Vec")

Training Node2Vec for 'Train Graph':
link_classification: using 'dot' method to combine node embeddings into edge embeddings
Epoch 1/6
10944/10944 - 42s - loss: 0.6327 - binary_accuracy: 0.6081
Epoch 2/6
10944/10944 - 55s - loss: 0.5730 - binary_accuracy: 0.6660
Epoch 3/6
10944/10944 - 52s - loss: 0.5263 - binary_accuracy: 0.7122
Epoch 4/6
10944/10944 - 44s - loss: 0.4552 - binary_accuracy: 0.7751
Epoch 5/6
10944/10944 - 43s - loss: 0.4061 - binary_accuracy: 0.8135
Epoch 6/6
10944/10944 - 44s - loss: 0.3843 - binary_accuracy: 0.8302

Best result with 'Node2Vec' embeddings from 'operator_l2'


,ROC AUC
name,
operator_hadamard,0.672112
operator_l1,0.753961
operator_l2,0.771417
operator_avg,0.710717


###### 4.2 Attri2Vec Graph Embedding and Link Prediction

In [64]:
from stellargraph.mapper import Attri2VecLinkGenerator, Attri2VecNodeGenerator
from stellargraph.layer import Attri2Vec


def attri2vec_embedding(graph, name):

    # Set the embedding dimension and walk number:
    dimension = [128]
    walk_number = 4

    print(f"Training Attri2Vec for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define an Attri2Vec training generator, which generates batches of training pairs
    generator = Attri2VecLinkGenerator(graph, batch_size)

    # Create the Attri2Vec model
    attri2vec = Attri2Vec(
        layer_sizes=dimension, generator=generator, bias=False, normalize=None
    )

    # Build the model and expose input and output sockets of Attri2Vec, for node pair inputs
    x_inp, x_out = attri2vec.in_out_tensors()

    # Use the link_classification function to generate the output of the Attri2Vec model
    prediction = link_classification(
        output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
    )(x_out)

    # Stack the Attri2Vec encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    model.fit(
        generator.flow(unsupervised_samples),
        epochs=epochs,
        verbose=2,
        use_multiprocessing=False,
        workers=1,
        shuffle=True,
    )

    # Build the model to predict node representations from node features with the learned Attri2Vec model parameters
    x_inp_src = x_inp[0]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

    # Get representations for all nodes in ``graph``
    node_gen = Attri2VecNodeGenerator(graph, batch_size).flow(graph_node_list)
    node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

In [65]:
attri2vec_result = train_and_evaluate(attri2vec_embedding, "Attri2Vec")

Training Attri2Vec for 'Train Graph':
link_classification: using 'ip' method to combine node embeddings into edge embeddings
Epoch 1/6
2189/2189 - 4s - loss: 0.7201 - binary_accuracy: 0.5393
Epoch 2/6
2189/2189 - 4s - loss: 0.7117 - binary_accuracy: 0.5614
Epoch 3/6
2189/2189 - 4s - loss: 0.7060 - binary_accuracy: 0.5781
Epoch 4/6
2189/2189 - 4s - loss: 0.7020 - binary_accuracy: 0.5861
Epoch 5/6
2189/2189 - 4s - loss: 0.7025 - binary_accuracy: 0.5877
Epoch 6/6
2189/2189 - 4s - loss: 0.7028 - binary_accuracy: 0.5876

Best result with 'Attri2Vec' embeddings from 'operator_hadamard'


,ROC AUC
name,
operator_hadamard,0.812228
operator_l1,0.630005
operator_l2,0.789152
operator_avg,0.810022


###### 4.3 GraphSAGE Graph Embedding and Link Prediction

In [66]:
from stellargraph.mapper import GraphSAGELinkGenerator, GraphSAGENodeGenerator
from stellargraph.layer import GraphSAGE


def graphsage_embedding(graph, name):

    # Set the embedding dimensions, the numbers of sampled neighboring nodes and walk number:
    dimensions = [128, 128]
    num_samples = [10, 5]
    walk_number = 1

    print(f"Training GraphSAGE for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define a GraphSAGE training generator, which generates batches of training pairs
    generator = GraphSAGELinkGenerator(graph, batch_size, num_samples)

    # Create the GraphSAGE model
    graphsage = GraphSAGE(
        layer_sizes=dimensions,
        generator=generator,
        bias=True,
        dropout=0.0,
        normalize="l2",
    )

    # Build the model and expose input and output sockets of GraphSAGE, for node pair inputs
    x_inp, x_out = graphsage.in_out_tensors()

    # Use the link_classification function to generate the output of the GraphSAGE model
    prediction = link_classification(
        output_dim=1, output_act="sigmoid", edge_embedding_method="ip"
    )(x_out)

    # Stack the GraphSAGE encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    model.fit(
        generator.flow(unsupervised_samples),
        epochs=epochs,
        verbose=2,
        use_multiprocessing=False,
        workers=4,
        shuffle=True,
    )

    # Build the model to predict node representations from node features with the learned GraphSAGE model parameters
    x_inp_src = x_inp[0::2]
    x_out_src = x_out[0]
    embedding_model = keras.Model(inputs=x_inp_src, outputs=x_out_src)

    # Get representations for all nodes in ``graph``
    node_gen = GraphSAGENodeGenerator(graph, batch_size, num_samples).flow(
        graph_node_list
    )
    node_embeddings = embedding_model.predict(node_gen, workers=1, verbose=0)

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

In [67]:
graphsage_result = train_and_evaluate(graphsage_embedding, "GraphSAGE")

Training GraphSAGE for 'Train Graph':
link_classification: using 'ip' method to combine node embeddings into edge embeddings
Epoch 1/6
548/548 - 25s - loss: 0.7036 - binary_accuracy: 0.5621
Epoch 2/6
548/548 - 24s - loss: 0.6806 - binary_accuracy: 0.5922
Epoch 3/6
548/548 - 24s - loss: 0.6739 - binary_accuracy: 0.6055
Epoch 4/6
548/548 - 24s - loss: 0.6698 - binary_accuracy: 0.6107
Epoch 5/6
548/548 - 24s - loss: 0.6597 - binary_accuracy: 0.6204
Epoch 6/6
548/548 - 24s - loss: 0.6545 - binary_accuracy: 0.6268


/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_

/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
/opt/anaconda3/lib/python3.8/site-packages/sklearn/linear_model/_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_


Best result with 'GraphSAGE' embeddings from 'operator_hadamard'


,ROC AUC
name,
operator_hadamard,0.895163
operator_l1,0.869408
operator_l2,0.893657
operator_avg,0.877777


4.4 GCN Embedding and Link Prediction

In [68]:
from stellargraph.mapper import FullBatchLinkGenerator, FullBatchNodeGenerator
from stellargraph.layer import GCN, LinkEmbedding


def gcn_embedding(graph, name):

    # Set the embedding dimensions and walk number:
    dimensions = [128, 128]
    walk_number = 1

    print(f"Training GCN for '{name}':")

    graph_node_list = list(graph.nodes())

    # Create the biased random walker to generate random walks
    walker = create_biased_random_walker(graph, walk_number, walk_length)

    # Create the unsupervised sampler to sample (target, context) pairs from random walks
    unsupervised_samples = UnsupervisedSampler(
        graph, nodes=graph_node_list, walker=walker
    )

    # Define a GCN training generator, which generates the full batch of training pairs
    generator = FullBatchLinkGenerator(graph, method="gcn")

    # Create the GCN model
    gcn = GCN(
        layer_sizes=dimensions,
        activations=["relu", "relu"],
        generator=generator,
        dropout=0.3,
    )

    # Build the model and expose input and output sockets of GCN, for node pair inputs
    x_inp, x_out = gcn.in_out_tensors()

    # Use the dot product of node embeddings to make node pairs co-occurring in short random walks represented closely
    prediction = LinkEmbedding(activation="sigmoid", method="ip")(x_out)
    prediction = keras.layers.Reshape((-1,))(prediction)
    #print(prediction)
    # Stack the GCN encoder and prediction layer into a Keras model, and specify the loss
    model = keras.Model(inputs=x_inp, outputs=prediction)
    model.compile(
        optimizer=keras.optimizers.Adam(lr=1e-3),
        loss=keras.losses.binary_crossentropy,
        metrics=[keras.metrics.binary_accuracy],
    )

    # Train the model
    batches = unsupervised_samples.run(batch_size)
    for epoch in range(epochs):
        print(f"Epoch: {epoch+1}/{epochs}")
        batch_iter = 1
        for batch in batches:
            samples = generator.flow(batch[0], targets=batch[1], use_ilocs=True)[0]
            [loss, accuracy] = model.train_on_batch(x=samples[0], y=samples[1])
            output = (
                f"{batch_iter}/{len(batches)} - loss:"
                + " {:6.4f}".format(loss)
                + " - binary_accuracy:"
                + " {:6.4f}".format(accuracy)
            )
            if batch_iter == len(batches):
                print(output)
            else:
                print(output, end="\r")
            batch_iter = batch_iter + 1

    # Get representations for all nodes in ``graph``
    embedding_model = keras.Model(inputs=x_inp, outputs=x_out)
    node_embeddings = embedding_model.predict(
        generator.flow(list(zip(graph_node_list, graph_node_list)))
    )
    node_embeddings = node_embeddings[0][:, 0, :]

    def get_embedding(u):
        u_index = graph_node_list.index(u)
        return node_embeddings[u_index]

    return get_embedding

In [69]:
gcn_result = train_and_evaluate(gcn_embedding, "GCN")

Training GCN for 'Train Graph':
Using GCN (local pooling) filters...
Epoch: 1/6
548/548 - loss: 0.6626 - binary_accuracy: 0.5000
Epoch: 2/6
548/548 - loss: 0.6839 - binary_accuracy: 0.5000
Epoch: 3/6
548/548 - loss: 0.6722 - binary_accuracy: 0.6000
Epoch: 4/6
548/548 - loss: 0.6968 - binary_accuracy: 0.5000
Epoch: 5/6
548/548 - loss: 0.6657 - binary_accuracy: 0.6000
Epoch: 6/6
548/548 - loss: 0.5933 - binary_accuracy: 0.5000

Best result with 'GCN' embeddings from 'operator_avg'


,ROC AUC
name,
operator_hadamard,0.891547
operator_l1,0.903348
operator_l2,0.896362
operator_avg,0.927841


###### 5 Conclusion

In [73]:
print('Node2Vec ROC AUC: ',0.771417)
print('Attri2Vec ROC AUC: ',0.812228)
print('GraphSAGE ROC AUC: ',0.895163)
print('GCN ROC AUC: ',0.927841)

Node2Vec ROC AUC:  0.771417
Attri2Vec ROC AUC:  0.812228
GraphSAGE ROC AUC:  0.895163
GCN ROC AUC:  0.927841


In [ ]:
####In sum,GCN+LR has the best performance in link prediction.

In [70]:
testdata = StellarGraph.to_networkx(graph_test)

In [71]:

testdata.nodes()

NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216, 217, 218, 219, 

In [72]:
####Here are tested edges for evaluation.
testdata.edges()

MultiEdgeDataView([(0, 3208), (1, 3209), (1, 3235), (1, 3220), (1, 2411), (2, 1596), (2, 3210), (2, 3209), (3, 3211), (3, 3212), (4, 3213), (4, 3221), (4, 3235), (4, 2423), (5, 3214), (5, 3215), (6, 3214), (6, 3208), (7, 3216), (8, 3211), (9, 3217), (9, 3218), (10, 3209), (10, 3219), (11, 3220), (11, 3221), (12, 3222), (12, 3214), (12, 3220), (12, 3258), (13, 3222), (14, 3223), (14, 3258), (15, 3223), (15, 855), (15, 2463), (16, 3223), (16, 3227), (16, 3379), (16, 878), (16, 855), (16, 2463), (16, 3226), (16, 246), (17, 3224), (18, 3220), (18, 3225), (18, 3224), (18, 859), (19, 3220), (19, 3224), (19, 870), (19, 1832), (19, 878), (19, 3270), (19, 241), (20, 3224), (21, 3225), (21, 3224), (21, 908), (21, 859), (21, 1320), (21, 3215), (22, 3226), (22, 3222), (22, 3258), (23, 3227), (23, 3226), (23, 864), (23, 2414), (23, 2429), (23, 262), (24, 3226), (24, 908), (24, 1217), (25, 3228), (25, 859), (25, 1320), (25, 878), (25, 3260), (25, 3277), (26, 3208), (26, 1832), (26, 2422), (26, 511),